In [0]:
!pip install tensorflowjs
!pip install tensorflow

In [1]:
# upload the model zip - Tensorflow version - from https://customvision.ai/projects
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())


files = upload_files()

#select the first file -> model zip
import zipfile
modelZip = files[0]

print(modelZip)

#unzip the model
import zipfile
zip = zipfile.ZipFile(modelZip)
zip.extractall('exportedModel')

filename = "./exportedModel/model.pb"
labels_filename = "./exportedModel/labels.txt"


Saving tf_model_pb.zip to tf_model_pb.zip
tf_model_pb.zip


In [37]:
import tensorflow as tf
import os  

def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

# load the graph pb  
graph = load_graph(filename)

'''
ATTENTION!!

To export a Tensorflow model to the javascript version or to work on it you should always know the 
name of the input node and of the output layer

But if you saved a .pb file from one of yours or external models you might not remember know those
info, to help you out here's a print of the tensors and operations

Code
| |
V V
  
with tf.Session(graph=graph) as sess:
    print(tf.contrib.graph_editor.get_tensors(tf.get_default_graph()))
    print(sess.graph.get_operations())


You should try to understand by looking at the description of the printed tensors or operation
with the specific size of the tensors.

The input layer in many cases it corresponds to the width/height of the images
your network can process

The output layer is related to the output of your network, it might be big as much as the number
of possible classes probabilities, or a single value for a boolean output or single class probabilities

'''


#remember that graph here is loaded in the function up there load_graph 
with tf.Session(graph=graph) as sess:
    print(tf.contrib.graph_editor.get_tensors(tf.get_default_graph()))
    print(sess.graph.get_operations())
    
    '''
    
    # printing the infos up there I think that those two names might be the right ones and then 
    # print them out to get a double check on them
    # remember that :0 mena :index or tensor in position [0] with that name
    
    input_node = 'Placeholder:0'
    output_layer = 'loss:0'
    
    x = sess.graph.get_tensor_by_name(input_node)
    y = sess.graph.get_tensor_by_name(output_layer)
    print( x )  
    print( y ) 
    
    # saving
    saver = tf.train.Saver()
  
    prob_tensor = sess.graph.get_tensor_by_name(output_layer)
    predictions, = sess.run(y, {input_node: [bgr_image] }) 
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)    
    '''

# <tf.Tensor 'prefix/Placeholder:0' shape=(?, 227, 227, 3) dtype=float32>
# <tf.Operation 'prefix/Placeholder' type=Placeholder>
# <tf.Tensor 'prefix/loss:0' shape=(?, 5) dtype=float32>
# <tf.Operation 'prefix/loss' type=Sigmoid>

[<tf.Operation 'prefix/Placeholder' type=Placeholder>, <tf.Operation 'prefix/data_bn/mean' type=Const>, <tf.Operation 'prefix/data_bn/mean/read' type=Identity>, <tf.Operation 'prefix/data_bn/variance' type=Const>, <tf.Operation 'prefix/data_bn/variance/read' type=Identity>, <tf.Operation 'prefix/data_bn/data_bn/add/y' type=Const>, <tf.Operation 'prefix/data_bn/data_bn/add' type=Add>, <tf.Operation 'prefix/data_bn/data_bn/Rsqrt' type=Rsqrt>, <tf.Operation 'prefix/data_bn/data_bn/mul' type=Mul>, <tf.Operation 'prefix/data_bn/data_bn/Neg' type=Neg>, <tf.Operation 'prefix/data_bn/data_bn/mul_1' type=Mul>, <tf.Operation 'prefix/data_bn/data_bn/add_1' type=Add>, <tf.Operation 'prefix/conv1/weights' type=Const>, <tf.Operation 'prefix/conv1/weights/read' type=Identity>, <tf.Operation 'prefix/conv1/Conv2D' type=Conv2D>, <tf.Operation 'prefix/conv1/biases' type=Const>, <tf.Operation 'prefix/conv1/biases/read' type=Identity>, <tf.Operation 'prefix/conv1/BiasAdd' type=BiasAdd>, <tf.Operation 'pref

In [0]:
# optimize the graph, reduce complexity 
python -m tensorflow.python.tools.optimize_for_inference --input=model.pb --output=optimized_graph.pb --input_names="Placeholder" --output_names="loss"

# quantize the network weights
python -m quantize_graph --input=optimized_graph.pb --output=rounded_graph.pb --output_node_names='loss' --mode=weights_rounded

# export to web format
tensorflowjs_converter --input_format=tf_frozen_model --output_node_names='loss' --saved_model_tags=serve ./optimized_graph.pb  ../js/assets

# the quantified model still doesn't work on Android and JS runtimes, just for future references or test
# tensorflowjs_converter --input_format=tf_frozen_model --output_node_names='loss' --saved_model_tags=serve ./rounded_graph.pb  ../js/assets

